<a href="https://colab.research.google.com/github/tensorflow-project/FineTuning/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/tensorflow-project/FineTuning
#%cd FineTuning

Cloning into 'FineTuning'...
remote: Enumerating objects: 424, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 424 (delta 37), reused 3 (delta 3), pack-reused 364
Receiving objects: 100% (424/424), 93.42 MiB | 14.00 MiB/s, done.
Resolving deltas: 100% (303/303), done.


In [ ]:
import sys
import os
from numpy import dot
from numpy.linalg import norm
from google.colab import drive
#from PIL import Image

py_file_location = "/content/FineTuning"
sys.path.append(os.path.abspath(py_file_location))
py_file_location = "/content/FineTuning/models"
sys.path.append(os.path.abspath(py_file_location))

import textual_inversion as txt

In [ ]:
sticker_embedding = []
cosine_similarity = []
broccoli = txt.get_embedding("broccoli")
cosine_similarity.append(txt.cosine_sim(broccoli, txt.get_embedding(txt.placeholder_token)))

In [ ]:
epochs = 5
epoch_num = list(range(epochs+1))
txt.training(epoch=epochs, model=txt.stable_diffusion, data= txt.train_ds)

In [ ]:
plt.plot(epoch_num, cosine_similarity)
plt.xlabel("Number of Epochs")
plt.ylabel("Cosine Similarity")
plt.title("Cosine Similarity between the basis and the new concept")
plt.show()

In [ ]:
txt.cosine_plot(epoch_num, cosine_similarity)

In [ ]:
drive.mount("/content/drive")
path = '/content/drive/MyDrive/angryweeeights.npy'

In [ ]:
###get the weights of the text encoder and save the to google drive
text_encoder_weights = np.array(stable_diffusion.text_encoder.get_weights())

### Save the weights array to a file on your Google Drive
np.save(path, text_encoder_weights)